In [1]:
import json

# Sample JSON documents
docs = [
    {
        "person": {
            "first_name": "Jane",
            "last_name": "Conner",
            "age": 2,
            "address": {
                "city": "MyCity3",
                "street": "Main St",
                "unique_key_1": 1000
            }
        },
    },
    {
        "person": {
            "first_name": "Apex",
            "last_name": "Predator",
            "age": 12,
            "address": {
                "city": "MyCity2",
                "street": "No St"
            }
        }
    },
    {
        "person": {
            "first_name": "Jane",
            "last_name": "Doe",
            "age": 9,
            "address": {
                "city": "MyCity",
                "street": "Main St"
            }
        }
    },
    {
        "person": {
            "first_name": "Sarah",
            "last_name": "Parker",
            "age": 15,
            "address": {
                "city": "Springfield and Miami",
                "street": "Main St"
            }
        }
    },
    {
        "person": {
            "first_name": "John",
            "last_name": "Doe",
            "age": 22,
            "address": {
                "city": "Springfield",
                "street": "Main St"
            }
        },
        "key1": "Value1",
        "summary": "This is a test summary."
    },
    {
        "person": {
            "first_name": "Alice",
            "last_name": "Smith",
            "age": 19,
            "address": {
                "city": "wood river",
                "street": "Elm St"
            }
        },
        "key1": "Value2",
        "summary": "Another test document."
    },
    {
        "person": {
            "first_name": "Allan",
            "last_name": "Brown",
            "age": 25,
            "address": {
                "city": "river wood",
                "street": "Oak St"
            }
        },
        "key1": "Value3",
        "summary": "Yet another test."
    }
]
print(json.dumps(docs, indent=4))


[
    {
        "person": {
            "first_name": "Jane",
            "last_name": "Conner",
            "age": 2,
            "address": {
                "city": "MyCity3",
                "street": "Main St",
                "unique_key_1": 1000
            }
        }
    },
    {
        "person": {
            "first_name": "Apex",
            "last_name": "Predator",
            "age": 12,
            "address": {
                "city": "MyCity2",
                "street": "No St"
            }
        }
    },
    {
        "person": {
            "first_name": "Jane",
            "last_name": "Doe",
            "age": 9,
            "address": {
                "city": "MyCity",
                "street": "Main St"
            }
        }
    },
    {
        "person": {
            "first_name": "Sarah",
            "last_name": "Parker",
            "age": 15,
            "address": {
                "city": "Springfield and Miami",
                "street": "Main St"
  

In [2]:
query_str = """
(and
  (contains * test)
  (exists key1)
  (or
    (< person.age 21)
    (startswith person.first_name "Al")
  )
  (contains person.** "wood river")
)
"""


query_ast = [
    "and",
    [
        "field", "*", ["contains", "test"]
    ],
    [
        "exists",
        "key1"
    ],
    [
        "or",
        [
            "field", "person.age", ["<", 21]
        ],
        [
            "field", "person.first_name", ["startswith", "Al"]
        ]
    ],
    [
        "field", "person.**", ["contains", "wood"]
    ]
]

query_ast = ["or",
                ["field", "person.age", ["<", 21]],
                ["field", "person.first_name", ["startswith", "Al"]],
                ["field", "person.address.city", ["contains", "Miami"]],
                ["field", "person.address.city", ["==", "MyCity"]],
                ["exists", "**.unique_key_1"]
            ]

import json
print("Query AST:")
print(json.dumps(query_ast, indent=2))


Query AST:
[
  "or",
  [
    "field",
    "person.age",
    [
      "<",
      21
    ]
  ],
  [
    "field",
    "person.first_name",
    [
      "startswith",
      "Al"
    ]
  ],
  [
    "field",
    "person.address.city",
    [
      "contains",
      "Miami"
    ]
  ],
  [
    "field",
    "person.address.city",
    [
      "==",
      "MyCity"
    ]
  ],
  [
    "exists",
    "**.unique_key_1"
  ]
]


In [3]:
from algebraic_search.fuzzy_json_query import FuzzyJsonQuery

# Create the query object
query = FuzzyJsonQuery(query_ast)
print("\nQuery:")
print(str(query))


Query:
(or (field person.age (< 21)) (field person.first_name (startswith Al)) (field person.address.city (contains Miami)) (field person.address.city (== MyCity)) (exists **.unique_key_1))


In [4]:
fuzzy_set = query.eval(docs)

node: ['or', ['field', 'person.age', ['<', 21]], ['field', 'person.first_name', ['startswith', 'Al']], ['field', 'person.address.city', ['contains', 'Miami']], ['field', 'person.address.city', ['==', 'MyCity']], ['exists', '**.unique_key_1']]
evaluating op='or' on operands=[['field', 'person.age', ['<', 21]], ['field', 'person.first_name', ['startswith', 'Al']], ['field', 'person.address.city', ['contains', 'Miami']], ['field', 'person.address.city', ['==', 'MyCity']], ['exists', '**.unique_key_1']] for doc={'person': {'first_name': 'Jane', 'last_name': 'Conner', 'age': 2, 'address': {'city': 'MyCity3', 'street': 'Main St', 'unique_key_1': 1000}}}
node: ['field', 'person.age', ['<', 21]]
evaluating op='field' on operands=['person.age', ['<', 21]] for doc={'person': {'first_name': 'Jane', 'last_name': 'Conner', 'age': 2, 'address': {'city': 'MyCity3', 'street': 'Main St', 'unique_key_1': 1000}}}
field_path='person.age' has field_values=[2]
evaluating expr_or_value=['<', 21] for value=2


In [5]:
# Print the degrees of membership
print(fuzzy_set.memberships)

for idx, degree in enumerate(fuzzy_set.memberships):
    print("\nDocument:")
    print(json.dumps(docs[idx], indent=2))
    print("Degree of membership:", degree)

[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Document:
{
  "person": {
    "first_name": "Jane",
    "last_name": "Conner",
    "age": 2,
    "address": {
      "city": "MyCity3",
      "street": "Main St",
      "unique_key_1": 1000
    }
  }
}
Degree of membership: 1.0

Document:
{
  "person": {
    "first_name": "Apex",
    "last_name": "Predator",
    "age": 12,
    "address": {
      "city": "MyCity2",
      "street": "No St"
    }
  }
}
Degree of membership: 0.0

Document:
{
  "person": {
    "first_name": "Jane",
    "last_name": "Doe",
    "age": 9,
    "address": {
      "city": "MyCity",
      "street": "Main St"
    }
  }
}
Degree of membership: 1.0

Document:
{
  "person": {
    "first_name": "Sarah",
    "last_name": "Parker",
    "age": 15,
    "address": {
      "city": "Springfield and Miami",
      "street": "Main St"
    }
  }
}
Degree of membership: 1.0

Document:
{
  "person": {
    "first_name": "John",
    "last_name": "Doe",
    "age": 22,
    "address": {
      "city": 